In [107]:
import pandas as pd
import re

In [180]:
icd10_full = pd.read_csv('/home/jovyan/work/data/ICD10pl.csv', encoding="Windows-1250", sep=';')
icd10_curl = pd.read_csv('/home/jovyan/work/data/ICD10_part_for_es.csv', sep=';') 

icd10_full['Kod podkategorii'] = icd10_full['Kod podkategorii'].apply(lambda x: x.strip())

In [201]:
icd10_curl

,icd10,text,text_search
0,E00.0,"Wrodzony zespół niedoboru jodu, typ neurologiczny","Wrodzony zespół niedoboru jodu, typ neurologiczny"
1,E00.1,"Wrodzony zespół niedoboru jodu, typ obrzęku śl...","Wrodzony zespół niedoboru jodu, typ obrzęku śl..."
2,E00.2,"Wrodzony zespół niedoboru jodu, typ mieszany","Wrodzony zespół niedoboru jodu, typ mieszany"
3,E00.9,"Wrodzony zespół niedoboru jodu, nieokreślony","Wrodzony zespół niedoboru jodu, nieokreślony"
4,E01.0,Rozlane (endemiczne) wole z powodu niedoboru jodu,Patologie tarczycy związane z niedoborem jodu ...
5,E01.1,Wieloguzkowe (endemiczne) wole z powodu niedob...,Patologie tarczycy związane z niedoborem jodu ...
6,E01.2,"Wole z powodu niedoboru jodu (endemiczne), nie...",Patologie tarczycy związane z niedoborem jodu ...
7,E01.8,Inne choroby tarczycy z powodu niedoboru jodu ...,Patologie tarczycy związane z niedoborem jodu ...
8,E02,Subkliniczna postać niedoczynności tarczycy z ...,Subkliniczna postać niedoczynności tarczycy z ...
9,E03.0,Wrodzona niedoczynność tarczycy z wolem rozlanym,"Niedoczynność tarczycy o innej etiologii, Wrod..."


In [183]:
def extendIcd10Array(icd10Input, icd10Extended):
    for i in icd10Input.index:
        #print(i)
        icd10Row = findFullFilledRow(icd10Extended, 'Kod podkategorii', icd10Input.at[i, 'icd10'])
        if icd10Row is not None:
            icd10Input.at[i, 'text_search'] = concatIcd10Item(icd10Row)
        else:
            icd10Input.at[i, 'text_search'] = icd10Input.at[i, 'text']
            
    return icd10Input

def findFullFilledRow(dfFull, column, value):
    df = dfFull.loc[dfFull[column] == value]
    if df.empty:
        return
    
    output = None
    for index, row in df.iterrows():
        if row['Poziom III'] != '|':
            output = row
        elif row['Poziom II'] != '|' and row['Poziom III'] == '|':
            output = row
        elif output is None and row['Poziom II'] == '|' and row['Poziom III'] == '|':
            output = row
    
    return output

def concatIcd10Item(icd10):
    output = concatIfNotDuplicated(icd10['Kategoria'], icd10['Podkategoria'])
    strings = [icd10['Poziom I'], icd10['Poziom II'], icd10['Poziom III']]
    
    for item in strings:
        output = concatIfNotEmpty(output, item)
    
    output = replaceBrackets(output)
    output =replaceNonAlphanumeric(output, ', -')
    output = output.replace('†', '')
    return output

def concatIfNotDuplicated(first, secound):
    if first not in secound:
        return concatIfNotEmpty(first, secound)
    else:
        return secound
        
def concatIfNotEmpty(start, suffix):
    suffix = suffix.strip()
    if start and suffix and suffix is not "|":
        return start + ', ' + suffix
    elif start is '' and suffix:
        return suffix
    else: 
        return start

def replaceBrackets(inputString):
    return re.sub('\(.*?\)', '', inputString)

def replaceNonAlphanumeric(inputString, exclude):
    return re.sub(r'[^\w'+ exclude +']', '', inputString)


In [192]:
icd10_curl_extended = extendIcd10Array(icd10_curl, icd10_full);
#display(icd10_curl_extended)

In [200]:
with open("/home/jovyan/work/data/ICD10_extended_part_for_es.txt", "w") as text_file:
    text_file.truncate(0) # need '0' when using r+
    for i, item in icd10_curl_extended.iterrows():
        out = '{"icd10": "' + item['icd10'] + '", "text": "' + item['text'] + '", "text_search": "' + item['text_search'] + '"}'
        text_file.write(out + "\n")